In [10]:
%pip install opencv-python mediapipe

  Using cached mediapipe-0.10.32-py3-none-win_amd64.whl.metadata (9.8 kB)
Using cached mediapipe-0.10.32-py3-none-win_amd64.whl (10.2 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Face Dec

In [12]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

model_path = "blaze_face_short_range.tflite"

base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.FaceDetectorOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO
)

detector = vision.FaceDetector.create_from_options(options)

cap = cv2.VideoCapture(0)
frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    detection_result = detector.detect_for_video(mp_image, frame_count)
    frame_count += 1

    for detection in detection_result.detections:
        bbox = detection.bounding_box
        x, y, w, h = bbox.origin_x, bbox.origin_y, bbox.width, bbox.height

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Face Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Face Landmark Dec

In [15]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

model_path = "face_landmarker.task"

base_options = python.BaseOptions(model_asset_path=model_path)

options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1
)

landmarker = vision.FaceLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)
frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    result = landmarker.detect_for_video(mp_image, frame_count)
    frame_count += 1

    if result.face_landmarks:
        for face_landmarks in result.face_landmarks:
            for landmark in face_landmarks:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

    cv2.imshow("Face Landmarks", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Drowsiness Detection

In [17]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

model_path = "face_landmarker.task"

base_options = python.BaseOptions(model_asset_path=model_path)

options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1
)

landmarker = vision.FaceLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)
frame_count = 0

def euclidean(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def get_eye_points(indices, landmarks, w, h):
    pts = []
    for i in indices:
        lm = landmarks[i]
        pts.append((int(lm.x * w), int(lm.y * h)))
    return pts

def compute_ear(points):
    vertical1 = euclidean(points[1], points[5])
    vertical2 = euclidean(points[2], points[4])
    horizontal = euclidean(points[0], points[3])
    return (vertical1 + vertical2) / (2.0 * horizontal)

# Left and Right eye indices
left_indices = [33, 160, 158, 133, 153, 144]
right_indices = [362, 385, 387, 263, 373, 380]

sleep_frame_counter = 0
SLEEP_THRESHOLD = 0.20
CONSEC_FRAMES = 15

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

    result = landmarker.detect_for_video(mp_image, frame_count)
    frame_count += 1

    if result.face_landmarks:
        face_landmarks = result.face_landmarks[0]
        h, w, _ = frame.shape

        left_eye = get_eye_points(left_indices, face_landmarks, w, h)
        right_eye = get_eye_points(right_indices, face_landmarks, w, h)

        left_ear = compute_ear(left_eye)
        right_ear = compute_ear(right_eye)

        ear = (left_ear + right_ear) / 2.0

        cv2.putText(frame, f"EAR: {ear:.2f}", (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw eye points
        for p in left_eye + right_eye:
            cv2.circle(frame, p, 2, (255, 0, 0), -1)

        # Sleep logic with consecutive frame check
        if ear < SLEEP_THRESHOLD:
            sleep_frame_counter += 1
        else:
            sleep_frame_counter = 0

        if sleep_frame_counter >= CONSEC_FRAMES:
            cv2.putText(frame, "SLEEPY", (30, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Sleep Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
